# 1. Assumptions

|Token Type|Description|Examples|
|:-|:-|:-|
|COMMA|The ',' character|,|
|PERIOD|The '.' character|.|
|Q_MARK|The '?' character|?|
|LEFT_PAREN|The '(' character|(|
|RIGHT_PAREN|The ')' character|)|
|COLON|The ':' character|	:|
|COLON_DASH|The string “:-”|:-|
|MULTIPLY|The '*' character|*|
|ADD|The '+' character|+|
|SCHEMES|The string “Schemes”|Schemes|
|FACTS|The string “Facts”|Facts|
|RULES|The string “Rules”|Rules|
|QUERIES|The string “Queries”|Queries|

In [1]:
import re

In [2]:
def getClauses(datalog_query):
    '''
    Description: Seperate each clause by character '.'
    type string: datalog_query
    rtype list
    '''
    return [item for item in datalog_query.split('.') if item != '']

In [3]:
def seperateHeadBody(clause):
    '''
    Description: get head and body for each clause
    type string: clauses
    rtype list
    '''
    head = clause.split(':-')[0]
    body = clause.split(':-')[1]
    return head,body

In [4]:
def getTermsInParens(a):
    '''
    Description: get each term in parentheses
    type string: a
    rtype list: results
    '''
    results = []
    if a.find('(') != -1 and a.find(')') != -1:
        terms = a[a.find("(")+1:a.find(")")].split(',')
        for term in terms:
            results.append(term)
    return results

In [5]:
def getColsFromHead(head):
    cols = getTermsInParens(head)
    return cols

In [6]:
def getPredPerTerm(term):
    predicate = ''
    if term.find('(') != -1 and term.find(')') != -1:
        predicate = term.split('(')[0]
    return predicate

In [7]:
def getTabsPerBody(body):
    tabs = []
    bodyLiterals = body.split(', ')
    for bodyLiteral in bodyLiterals:
        if bodyLiteral.find('(') != -1 and bodyLiteral.find(')') != -1:
            predicate = bodyLiteral.split('(')[0]
            tabs.append(predicate)
    return tabs

class Node(object):
    def __init__(self, data):
        self.data = data
        self.children = []

    def add_child(self, obj):
        self.children.append(obj)

def parseDatalog2Tree(clauses):
    '''
    type list[string]: clauses
    rtype Node: n
    '''
    n = Node(seperateHeadBody(clauses[0])[0])
    for literal in seperateHeadBody(clauses[0])[1].split(', '):
        n.add_child(literal)
    return n

In [8]:
def mapFactSchema(factList, inputSchemas):
    '''
    Description: make a dictionary to contain the fact schama and corresponds fact
    type list: factList
    type list: inputSchema
    '''
    results = {}
    for fact in factList:
        pred = getPredPerTerm(fact)
        for schema in inputSchemas:
            if schema.find(pred) != -1:
                for i in range(len(getTermsInParens(schema))):
                    key = pred + "." + getTermsInParens(schema)[i]
                    if key in results:
                        results[key].append(getTermsInParens(fact)[i])
                    else:
                        results[key] = [getTermsInParens(fact)[i]]
    return results

In [9]:
def getSchemaDict(inputSchemas):
    '''
    Description: Create a dictionary for schemas
    type list: inputSchemas
    rtype dictionary
    '''
    schemaDict = {}
    for schema in inputSchemas:
        pred = getPredPerTerm(schema)
        if pred in schemaDict:  
            schemaDict[pred] += getTermsInParens(schema)
        else:
            schemaDict[pred] = getTermsInParens(schema)
    return schemaDict

def translateDatalog2Sql(n, factList, inputSchemas):
    '''
    type Node: n
    type list: inputSchemas
    type list: factList
    rtype sqlQuery: string
    '''
    schemaDict = getSchemaDict(inputSchemas)
    factSchemaDict = mapFactSchema(factList, inputSchemas)
    sqlQuery = ''
    sqlQueryFrom = 'From '
    sqlQueryWhere = 'Where '
    sqlQuerySelect = 'Select '
    duplicateTerms = {}
    head = getPredPerTerm(n.data)
    sqlQueryView = 'Create View ' + head + ' as('
    headTerms = getTermsInParens(n.data)
    colNums = len(headTerms)
    i = 1
    for children in n.children:
        tab = getPredPerTerm(children)
        sqlQueryFrom += tab + ', '
        terms = getTermsInParens(children)
        # Update where clause
        for termPos in range(len(terms)):
            termCol = tab + '.' + terms[termPos]
            if termCol not in factSchemaDict:
                ### Need to edit
                sqlQueryWhere += tab + '.' + schemaDict[tab][termPos] + '=' + terms[termPos]  +" and "
            if terms[termPos] not in duplicateTerms:
                duplicateTerms[terms[termPos]] = termCol
            else:
                sqlQueryWhere += duplicateTerms[terms[termPos]] + "=" + termCol + " and "
        # Update select clause
        for term in terms:        
            if term in headTerms and i <= 2:
                sqlQuerySelect += termCol + ", "
                i += 1

    sqlQuery += sqlQueryView + "\n" + sqlQueryFrom.strip(', ') + "\n" + sqlQueryWhere.strip(' and ') + "\n" + sqlQuerySelect.strip(', ') + ')'
    return sqlQuery

In [58]:
def getBodyDict(body):
    '''
    Description: create a dictionary to contain predicate-term pair
    type string: body
    rtype dict: bodyDict
    '''
    bodyDict = {}
    bodyLiterals = body.split(', ')
    for bodyLiteral in bodyLiterals:
        if bodyLiteral.find('(') != -1 and bodyLiteral.find(')') != -1:
            predicate = bodyLiteral.split('(')[0]
            bodyDict[predicate] = bodyLiteral[bodyLiteral.find('(')+1:bodyLiteral.find(')')].split(',')
        else:
            bodyDict[bodyLiteral] = bodyLiteral
    return bodyDict

In [11]:
def getSelectQuery(cols, bodyDict):
    '''
    Description: generate clause
    type list[string]: cols
    type dict: bodyDict
    rtype string: selectCaluse
    '''
    selectClause = 'select '
    for col in cols:
        for pred in bodyDict.keys():
            if col in bodyDict[pred]:
                selectClause += pred + '.' + col + ', '
                break
    selectClause = selectClause.strip(', ')
    return selectClause

In [59]:
def getFromQuery(body):
    '''
    Descrption: generate from clause
    type string: body
    rtype string: fromClause
    '''
    fromClause = 'from '
    for tab in getTabsPerBody(body):
        fromClause += tab + ', '
    fromClause = fromClause.strip(', ')
    return fromClause

In [13]:
def getWhereQuery(bodyDict, schemaDict):
    '''
    Description: generate where clause
    type dict: bodyDict
    type dict: schemaDict
    rtype string: whereClause
    '''
    split_comp = ">|<|="
    dictCol = {}
    whereClause = 'where '
    for pred in bodyDict.keys():
        if len(re.findall(split_comp, pred))==0:
            for term in bodyDict[pred]:
                if term not in dictCol and term in schemaDict[pred]:
                    dictCol[term] = pred
                elif term in dictCol and term in schemaDict[pred]:
                    whereClause += dictCol[term] + '.' + term + '=' + pred + '.' + term + ' and '
                # specify constant
                elif term not in schemaDict[pred]:
                    pos = bodyDict[pred].index(term)
                    whereClause += pred + '.'+ schemaDict[pred][pos] + '=' + term + ' and '
        # translate built-in predicate
        else:
            term = re.split(split_comp,pred)[0]
            term = term.strip(' ')
            whereClause += dictCol[term] + '.' + pred + ' and '
    whereClause = whereClause.strip(' and ')
    return whereClause 

In [37]:
inp = 'r1(a,b):-fact1(x,a), fact2(20,y), fact3(y,b), x >= 20.'

In [38]:
factList =['fact1(2,1)','fact1(2,3)','fact1(2,4)','fact2(20,10)','fact2(20,30)','fact2(20,40)', 'fact3(22,21)','fact3(22,23)','fact3(22,24)']

In [39]:
inputSchemas = 'fact1(x,a), fact2(a,y), fact3(y,b)'

In [40]:
clauses = getClauses(inp)

In [41]:
clauses

['r1(a,b):-fact1(x,a), fact2(20,y), fact3(y,b), x >= 20']

In [42]:
head = seperateHeadBody(clauses[0])[0]

In [43]:
head

'r1(a,b)'

In [44]:
cols = getColsFromHead(head)

In [45]:
cols

['a', 'b']

In [46]:
body = seperateHeadBody(clauses[0])[1]

In [47]:
body

'fact1(x,a), fact2(20,y), fact3(y,b), x >= 20'

In [48]:
bodyDict = getBodyDict(body)

In [49]:
bodyDict

{'fact1': ['x', 'a'],
 'fact2': ['20', 'y'],
 'fact3': ['y', 'b'],
 'x >= 20': 'x >= 20'}

In [50]:
selectClause =  getSelectQuery(cols, bodyDict)

In [51]:
selectClause

'select fact1.a, fact3.b'

In [52]:
fromClause = getFromQuery(body)

In [53]:
schemaDict = getBodyDict(inputSchemas)

In [54]:
whereClause = getWhereQuery(bodyDict, schemaDict)

In [55]:
whereClause

'where fact2.a=20 and fact2.y=fact3.y and fact1.x >= 20'

In [56]:
sql = selectClause + ' ' + fromClause + ' ' + whereClause

In [57]:
sql

'select fact1.a, fact3.b from fact1, fact2, fact3 where fact2.a=20 and fact2.y=fact3.y and fact1.x >= 20'